# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


## 4. CoreNLP Analysis

We first load data files and download the pre-processed dataframes. 

In [72]:
from load_data import *
from coreNLP_analysis import *

download_data()
plot_df = load_plot_df()
movie_df = load_movie_df()
char_df = load_char_df()
names_df = load_names_df()
cluster_df = load_cluster_df()

### 4.1. Extracting characters

For any character, we want to extract related information (from name clusters, character metadata) as well as actions, characteristics and relations (from CoreNLP). We first extract information from the pre-processed dataframes. 

We will use Harry Potter's character as an example

In [152]:
char_name = 'Harry Potter'
movie_ids = list(char_df[char_df['Character name'] == 'Harry Potter']['Wikipedia ID'])
char_ids = names_df.loc[char_name].values[0]
trope = cluster_df.loc[cluster_df['Character name'] == char_name]
# if trop is empty, set trope to None
if trope.empty:
    trope = None

print('Movies with character', char_name, ':')
print('\tMovie IDs:', movie_ids)
print('\tCharacter IDs:', char_ids)
print('\tTrope:', trope)

movie_id = movie_ids[3] 
print('Selecting movie ID as example:', movie_id)

Movies with character Harry Potter :
	Movie IDs: [858575, 667372, 670407, 31941988, 9834441, 667368, 667371, 667361, 667361]
	Character IDs: ['/m/0jz6jt', '/m/02tbbh6', '/m/0jz6mq', '/m/0jz6hs', '/m/02tbf6n', '/m/0jz6b0', '/m/0jz6dz', '/m/09lybcb']
	Trope: None
Selecting movie ID as example: 31941988


We now extract information from the CoreNLP plot summary analysis. Each xml file has a tree structure detailing each word of each sentence as well as the parsed sentence in tree form. We extract all parsed sentences from the xml file, each of which we can view as a tree structure. 

In [169]:
tree = get_tree(movie_id)
parsed_str = get_parsed_sentences(tree)[5]
print(parsed_str)
print_tree(parsed_str)

(ROOT (S (PP (IN In) (NP (NP (NNP Bellatrix) (POS 's)) (NN vault))) (, ,) (NP (NNP Harry)) (VP (VBZ discovers) (SBAR (S (NP (DT the) (NNP Horcrux)) (VP (VBZ is) (NP (NP (NNP Helga) (NNP Hufflepuff) (POS 's)) (NN cup)))))) (. .))) 
                                                ROOT                                                 
                                                 |                                                    
                                                 S                                                   
                _________________________________|_________________________________________________   
               |             |    |                               VP                               | 
               |             |    |        _______________________|____                            |  
               |             |    |       |                           SBAR                         | 
               |             |    |       |         

We also want to extract all character names from the xml file. Note that we aggregate consecutive words tagged as NNP (noun, proper, singular) as the same character name (this assumes that plot summaries never contain two distinct names side by side without delimiting punctuation). This is a reasonable assumption since list of names are almost always separated by commas. 

In [170]:
characters = get_characters(tree)
characters[:15]

['Voldemort',
 'Albus Dumbledore',
 'Severus Snape',
 'Dobby',
 'Harry Potter',
 'Ron',
 'Hermione',
 'Griphook',
 'Harry',
 'Ollivander',
 'Ollivander',
 'Draco Malfoy',
 'Malfoy',
 'Harry',
 'Harry']

Notice that some characters are sometimes mentioned by their full name, and sometimes by a partial name (e.g. Harry Potter is most often mentioned as simply Harry). To get a more precise idea of how many times each character is mentioned, we wish to denote each character by their full name, i.e. the longest version of their name that appears in the plot summary. 

To optimize full name lookup, for each plot summary we construct a dictionary which stores as key every partial name mentioned, and as corresponding values the full name of each character.  

In [171]:
char_name = 'Albus'
full_name = get_full_name(char_name, characters)
print('Example: the full name of "{}" is "{}".'.format(char_name,full_name))
print('Full name dictionary:', full_name_dict(characters))

Example: the full name of "Albus" is "Albus Dumbledore".
Full name dictionary: {'Voldemort': 'Voldemort', 'Albus Dumbledore': 'Albus Dumbledore', 'Severus Snape': 'Severus Snape', 'Dobby': 'Dobby', 'Harry Potter': 'Harry Potter', 'Ron': 'Ron', 'Hermione': 'Hermione Weasley', 'Griphook': 'Griphook', 'Harry': 'Harry Potter', 'Ollivander': 'Ollivander', 'Draco Malfoy': 'Draco Malfoy', 'Malfoy': 'Draco Malfoy', 'Helga Hufflepuff': 'Helga Hufflepuff', 'Rowena Ravenclaw': 'Rowena Ravenclaw', 'Hogsmeade': 'Hogsmeade', 'Aberforth Dumbledore': 'Aberforth Dumbledore', 'Ariana': 'Ariana', 'Neville Longbottom': 'Neville Longbottom', 'Snape': 'Severus Snape', 'Minerva McGonagall': 'Minerva McGonagall', 'Luna Lovegood': 'Luna Lovegood', 'Helena Ravenclaw': 'Helena Ravenclaw', 'Gregory Goyle': 'Gregory Goyle', 'Blaise Zabini': 'Blaise Zabini', 'Nagini': 'Nagini', 'Fred': 'Fred', 'Lily': 'Lily', 'James': 'James', 'Dumbledore': 'Albus Dumbledore', 'Neville': 'Neville Longbottom', 'Molly Weasley': 'Moll


From the list of character full names, we can now construct a dictionary with keys being the characters' full name and values being the number of times any version of their name is mentioned. 

In [172]:
aggregate_characters(characters)

{'Voldemort': 21,
 'Albus Dumbledore': 5,
 'Severus Snape': 11,
 'Dobby': 1,
 'Harry Potter': 26,
 'Ron': 6,
 'Hermione Weasley': 6,
 'Griphook': 3,
 'Ollivander': 2,
 'Draco Malfoy': 3,
 'Helga Hufflepuff': 1,
 'Rowena Ravenclaw': 1,
 'Hogsmeade': 1,
 'Aberforth Dumbledore': 1,
 'Ariana': 1,
 'Neville Longbottom': 3,
 'Minerva McGonagall': 1,
 'Luna Lovegood': 1,
 'Helena Ravenclaw': 1,
 'Gregory Goyle': 1,
 'Blaise Zabini': 1,
 'Nagini': 3,
 'Fred': 1,
 'Lily': 2,
 'James': 1,
 'Molly Weasley': 1,
 'Ginny Potter': 1}

We can now extract the most mentioned characters in any plot summary, in descending order of frequency. 

In [173]:
# Given a parse tree, extract the most mentioned characters in decreasing order
def most_mentioned(movie_id):
    '''
    Input: 
        tree: parse tree of the xml file
        N: the number of characters to return
    Output:
        A dictionary of the N characters most mentioned in the movie
    '''
    tree = get_tree(movie_id)
    characters = get_characters(tree)
    character_dict = aggregate_characters(characters)
    sorted_characters = sorted(character_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_characters

We can then see that Harry Potter is indeed the main character of the movie, as he is mentioned 26 times, more than any other character in the summary. 

In [174]:
most_mentioned(movie_id)[:10]

[('Harry Potter', 26),
 ('Voldemort', 21),
 ('Severus Snape', 11),
 ('Ron', 6),
 ('Hermione Weasley', 6),
 ('Albus Dumbledore', 5),
 ('Griphook', 3),
 ('Draco Malfoy', 3),
 ('Neville Longbottom', 3),
 ('Nagini', 3)]

 ### 4.2. Extracting relationships

 We cannot extract character interactions directly from the CoreNLP output (or can we?). Instead, we use the number of common mentions of two characters in the same sentence as a proxy for the number of interactions. For any movie, we find the number of common mentions (i.e. interactions) for each pair of characters. 

In [177]:
character_pairs(movie_id)[:10]

[(('Hermione Weasley', 'Ron'), 4),
 (('Harry Potter', 'Voldemort'), 4),
 (('Albus Dumbledore', 'Voldemort'), 3),
 (('Albus Dumbledore', 'Severus Snape'), 2),
 (('Harry Potter', 'Hermione Weasley'), 2),
 (('Harry Potter', 'Ron'), 2),
 (('Nagini', 'Voldemort'), 2),
 (('Harry Potter', 'Lily'), 2),
 (('Albus Dumbledore', 'Harry Potter'), 2),
 (('Severus Snape', 'Voldemort'), 1)]

In [179]:
main_interaction = character_pairs(movie_id)[0][0]
main_interaction

('Hermione Weasley', 'Ron')

### 4.3. Extracting main character and interactions

We will now use the above code to obtain the main character and most important pair of characters for every plot summary. 

In [180]:
pairs_df = plot_df
pairs_df['Main character'] = pairs_df['Wikipedia ID'].apply(lambda x: most_mentioned(x)[0])
pairs_df['Main interaction'] = pairs_df['Wikipedia ID'].apply(lambda x: character_pairs(x)[0][0])

IndexError: list index out of range

In [ ]:
# Merge pairs dataset with characters 
char_df['Wikipedia ID'] = char_df['Wikipedia ID'].astype(str)
pairs_df['Wikipedia ID'] = pairs_df['Wikipedia ID'].astype(str)
pairs_char = pairs_df.merge(char_df, on="Wikipedia ID")

# Filter out the nan values
pairs_char = pairs_char[~pairs_char['Character name'].isna()]

# Create columns which indicates if char1 and char2 are in character name 
pairs_char['char1_is_in_name'] = pairs_char.apply(lambda x: 1 if x['char1'] in x['Character name'] else 0, axis=1)
pairs_char['char2_is_in_name'] = pairs_char.apply(lambda x: 1 if x['char2'] in x['Character name'] else 0, axis=1)
pairs_char[pairs_char['char1_is_in_name'] == 1 & pairs_char['char2_is_in_name'] == 1]

### 4.4. CoreNLP Analysis

To use the powerful CoreNLP model, first [download it](https://stanfordnlp.github.io/CoreNLP/download.html), then cd into the downloaded `stanford-corenlp` directory. If you have Java, you can run the following command to open a CoreNLP shell: 


`java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer` 
 
Now that the shell is running, we can use their models to annotate some sentences. 

In [ ]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')
nlp.annotate('Barack Obama was born in Hawaii.  He is the president. Obama was elected in 2008.')

